In [15]:
import pandas as pd
import numpy as np 
ESG=pd.read_csv('../data/ESG_Score.csv')
hist_scope=pd.read_csv('../data/raw/hist_scope.csv')
hist_score=pd.read_csv('../data/raw/hist_score.csv')
merged_data=pd.read_csv('../data/merged_data.csv')

In [16]:
hist_score

,Isin,Year,SR001,SR002,SR003,SR004
0,US0079031078,2018.0,6.070000172,6.841240406,4.237225056,7.072086811
1,US0326541051,2018.0,4.53000021,4.951658726,2.288488626,6.702093601
2,US0382221051,2018.0,6.900000095,6.190028667,6.683382988,8.140346527
3,US11135F1012,2018.0,2.029999971,0,1.427484035,7.473958492
4,US29355A1079,2018.0,3.069999933,0.130537882,6.115842342,5.971273899
...,...,...,...,...,...,...
4739,NaN,NaN,NaN,NaN,NaN,NaN
4740,NaN,NaN,NaN,NaN,NaN,NaN
4741,NaN,NaN,NaN,NaN,NaN,NaN
4742,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
hist_score = hist_score.replace(['#N/A', '#N/A Field Not Applicable'], np.nan)
numeric_columns = ['SR001', 'SR002', 'SR003', 'SR004']
hist_score[numeric_columns] = hist_score[numeric_columns].astype(float)
hist_score

,Isin,Year,SR001,SR002,SR003,SR004
0,US0079031078,2018.0,6.07,6.841240,4.237225,7.072087
1,US0326541051,2018.0,4.53,4.951659,2.288489,6.702094
2,US0382221051,2018.0,6.90,6.190029,6.683383,8.140347
3,US11135F1012,2018.0,2.03,0.000000,1.427484,7.473958
4,US29355A1079,2018.0,3.07,0.130538,6.115842,5.971274
...,...,...,...,...,...,...
4739,NaN,NaN,NaN,NaN,NaN,NaN
4740,NaN,NaN,NaN,NaN,NaN,NaN
4741,NaN,NaN,NaN,NaN,NaN,NaN
4742,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
new_rows = []
for index, row in ESG.iterrows():
    isin = row['ISIN']
    esg_score = row['ESG SCORE']
    env_score = row['ENVIRONMENTAL SCORE']
    social_score = row['SOCIAL SCORE']
    governance_score = row['GOVERNANCE SCORE']
    
    new_row = {
        'Isin': isin,
        'Year': 2023,
        'SR001': esg_score,
        'SR002': env_score,
        'SR003': social_score,
        'SR004': governance_score
    }
    new_rows.append(new_row)

new_rows_df = pd.DataFrame(new_rows)
hist_score = pd.concat([hist_score, new_rows_df], ignore_index=True)


In [19]:
hist_score.head()

,Isin,Year,SR001,SR002,SR003,SR004
0,US0079031078,2018.0,6.07,6.841240,4.237225,7.072087
1,US0326541051,2018.0,4.53,4.951659,2.288489,6.702094
2,US0382221051,2018.0,6.90,6.190029,6.683383,8.140347
3,US11135F1012,2018.0,2.03,0.000000,1.427484,7.473958
4,US29355A1079,2018.0,3.07,0.130538,6.115842,5.971274


In [20]:

hist_2021 = hist_score[hist_score['Year'] == 2021]

combined = pd.merge(hist_2021[['Isin', 'SR001', 'SR002', 'SR003', 'SR004']], 
                    ESG[['ISIN', 'ESG SCORE', 'ENVIRONMENTAL SCORE', 'SOCIAL SCORE', 'GOVERNANCE SCORE']], 
                    left_on='Isin', right_on='ISIN', how='left')

combined['Trend_ESG_SR001'] = combined['ESG SCORE'] - combined['SR001']
combined['Trend_ENV_SR002'] = combined['ENVIRONMENTAL SCORE'] - combined['SR002']
combined['Trend_SOC_SR003'] = combined['SOCIAL SCORE'] - combined['SR003']
combined['Trend_GOV_SR004'] = combined['GOVERNANCE SCORE'] - combined['SR004']

historical_score = pd.merge(hist_score, combined[['Isin', 'Trend_ESG_SR001', 'Trend_ENV_SR002', 'Trend_SOC_SR003', 'Trend_GOV_SR004']], on='Isin', how='left')

historical_score

,Isin,Year,SR001,SR002,SR003,SR004,Trend_ESG_SR001,Trend_ENV_SR002,Trend_SOC_SR003,Trend_GOV_SR004
0,US0079031078,2018.0,6.070000,6.841240,4.237225,7.072087,0.190441,0.230901,0.417430,-0.192367
1,US0326541051,2018.0,4.530000,4.951659,2.288489,6.702094,0.866660,1.043274,0.214858,0.810937
2,US0382221051,2018.0,6.900000,6.190029,6.683383,8.140347,0.656745,0.230856,1.174702,0.244051
3,US11135F1012,2018.0,2.030000,0.000000,1.427484,7.473958,0.660284,1.069467,0.703850,-0.617206
4,US29355A1079,2018.0,3.070000,0.130538,6.115842,5.971274,-0.749523,0.814302,-4.199872,0.463123
...,...,...,...,...,...,...,...,...,...,...
5291,NO0012916131,2023.0,4.884967,4.932675,3.435348,6.270973,0.914967,1.743519,0.297694,0.141069
5292,US00676P1075,2023.0,3.009972,1.260000,0.752634,7.600606,0.629972,0.254923,0.057872,-0.245758
5293,IT0005433740,2023.0,2.894523,4.619199,1.023712,2.465766,0.764523,0.803481,0.004565,0.191609
5294,CNE0000001Y0,2023.0,1.717311,0.000000,0.000000,5.724370,0.497311,0.000000,0.000000,-0.205604


In [21]:
historical_score.to_csv('../data/historical_score.csv', index=False)

In [22]:
hist_scope = hist_scope.replace(['#N/A', '#N/A Field Not Applicable'], np.nan)
numeric_columns = ['EG017', 'EG018', 'EG030', 'IS010']
hist_scope[numeric_columns] = hist_scope[numeric_columns].astype(float)
hist_scope

,Isin,Year,EG017,EG018,EG030,IS010
0,US0079031078,2010,26.628,140.384,286.840,6494.000000
1,US0326541051,2010,34.933,96.687,14.211,2761.503000
2,US0382221051,2010,42.172,133.960,36.863,9549.000000
3,US11135F1012,2010,110.810,156.699,10.771,2093.000000
4,US29355A1079,2010,0.005,9.199,14.247,61.661000
...,...,...,...,...,...,...
7723,NO0012916131,2023,0.016,0.066,9.068,1089.491000
7724,US00676P1075,2023,8.514,3.846,2.522,388.788000
7725,IT0005433740,2023,1.665,1.107,0.909,91.023833
7726,CNE0000001Y0,2023,7.336,6.103,2.842,2858.122182


In [23]:
hist_scope=pd.merge(hist_scope, merged_data[['ISIN', 'Industry']], left_on='Isin', right_on='ISIN', how='left')

In [24]:
hist_scope

,Isin,Year,EG017,EG018,EG030,IS010,ISIN,Industry
0,US0079031078,2010,26.628,140.384,286.840,6494.000000,US0079031078,Semiconductors
1,US0326541051,2010,34.933,96.687,14.211,2761.503000,US0326541051,Semiconductors
2,US0382221051,2010,42.172,133.960,36.863,9549.000000,US0382221051,Semiconductors
3,US11135F1012,2010,110.810,156.699,10.771,2093.000000,US11135F1012,Semiconductors
4,US29355A1079,2010,0.005,9.199,14.247,61.661000,US29355A1079,Semiconductors
...,...,...,...,...,...,...,...,...
7723,NO0012916131,2023,0.016,0.066,9.068,1089.491000,NO0012916131,Software and Tech Services
7724,US00676P1075,2023,8.514,3.846,2.522,388.788000,US00676P1075,Software and Tech Services
7725,IT0005433740,2023,1.665,1.107,0.909,91.023833,IT0005433740,Software and Tech Services
7726,CNE0000001Y0,2023,7.336,6.103,2.842,2858.122182,CNE0000001Y0,Software and Tech Services


In [25]:
hist_scope['Sum_EG'] = (hist_scope['EG017'] + hist_scope['EG018'])/hist_scope['IS010']*1000


# Calcular los percentiles 25, 50 y 75 por Industry y Year
percentiles = hist_scope.groupby(['Industry', 'Year'])['Sum_EG'].quantile([0.25, 0.5, 0.75]).unstack()

# Renombrar las columnas de percentiles
percentiles.columns = [f'Sum_EG_{int(pct*100)}' for pct in percentiles.columns]

# Resetear el índice para poder hacer el merge
percentiles = percentiles.reset_index()

# Hacer el merge de los percentiles con el DataFrame original
hist_scope = hist_scope.merge(percentiles, on=['Industry', 'Year'], how='left')

In [26]:
hist_scope

,Isin,Year,EG017,EG018,EG030,IS010,ISIN,Industry,Sum_EG,Sum_EG_25,Sum_EG_50,Sum_EG_75
0,US0079031078,2010,26.628,140.384,286.840,6494.000000,US0079031078,Semiconductors,25.717893,0.836169,7.411292,33.869677
1,US0326541051,2010,34.933,96.687,14.211,2761.503000,US0326541051,Semiconductors,47.662450,0.836169,7.411292,33.869677
2,US0382221051,2010,42.172,133.960,36.863,9549.000000,US0382221051,Semiconductors,18.445073,0.836169,7.411292,33.869677
3,US11135F1012,2010,110.810,156.699,10.771,2093.000000,US11135F1012,Semiconductors,127.811276,0.836169,7.411292,33.869677
4,US29355A1079,2010,0.005,9.199,14.247,61.661000,US29355A1079,Semiconductors,149.267771,0.836169,7.411292,33.869677
...,...,...,...,...,...,...,...,...,...,...,...,...
7723,NO0012916131,2023,0.016,0.066,9.068,1089.491000,NO0012916131,Software and Tech Services,0.075265,0.255928,2.110228,21.872761
7724,US00676P1075,2023,8.514,3.846,2.522,388.788000,US00676P1075,Software and Tech Services,31.791105,0.255928,2.110228,21.872761
7725,IT0005433740,2023,1.665,1.107,0.909,91.023833,IT0005433740,Software and Tech Services,30.453563,0.255928,2.110228,21.872761
7726,CNE0000001Y0,2023,7.336,6.103,2.842,2858.122182,CNE0000001Y0,Software and Tech Services,4.702038,0.255928,2.110228,21.872761


In [27]:
hist_scope = hist_scope.drop(columns=['ISIN', 'Sum_EG'])
hist_scope

,Isin,Year,EG017,EG018,EG030,IS010,Industry,Sum_EG_25,Sum_EG_50,Sum_EG_75
0,US0079031078,2010,26.628,140.384,286.840,6494.000000,Semiconductors,0.836169,7.411292,33.869677
1,US0326541051,2010,34.933,96.687,14.211,2761.503000,Semiconductors,0.836169,7.411292,33.869677
2,US0382221051,2010,42.172,133.960,36.863,9549.000000,Semiconductors,0.836169,7.411292,33.869677
3,US11135F1012,2010,110.810,156.699,10.771,2093.000000,Semiconductors,0.836169,7.411292,33.869677
4,US29355A1079,2010,0.005,9.199,14.247,61.661000,Semiconductors,0.836169,7.411292,33.869677
...,...,...,...,...,...,...,...,...,...,...
7723,NO0012916131,2023,0.016,0.066,9.068,1089.491000,Software and Tech Services,0.255928,2.110228,21.872761
7724,US00676P1075,2023,8.514,3.846,2.522,388.788000,Software and Tech Services,0.255928,2.110228,21.872761
7725,IT0005433740,2023,1.665,1.107,0.909,91.023833,Software and Tech Services,0.255928,2.110228,21.872761
7726,CNE0000001Y0,2023,7.336,6.103,2.842,2858.122182,Software and Tech Services,0.255928,2.110228,21.872761


In [28]:
historical_scope= hist_scope
historical_scope.to_csv('../data/historical_scope.csv', index=False)